In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import pandas as pd
import time
from urllib.request import urlopen
import numpy as np
from datetime import timedelta, datetime, date
import requests
import re
import json
from ast import literal_eval
from io import StringIO

In [2]:
"""
DCF (현금흐름할인) -- DCF = CF1/(1+r)^1 + CF2/(1+r)^2 + CF3/(1+r)^3 ...+ CFn/(1+r)^n
    FCFF -- CFO + INT(1-Tax Rate) - CAPEX
    r (할인율) -- 자본/자산 * Equity rate + 부채/자산*(이자율(1-t))
    CAPM rate(Equity rate) -- Re = Rf + [Ba x (Rm - Rf)]
    g (GDP 성장율) -- 현대 증권 경기 예측 보고서 참조
"""


#고정 변수
#I_Growth = Terminal Value계산시 쓰는 성장률
#GDP 성장률 
I_Growth = 0.02 # 현대증권 경제 보고서 참조(파일명: 현대증권경기예측.pdf //8 p.g " 국내 잠재성장률은 2016~2020년 중 2%대 진입이 예상되며... ")
GDPsite=pd.read_html('http://www.index.go.kr/potal/main/EachDtlPageDetail.do?idx_cd=2736', header=0)
GDPgrowthRate=GDPsite[1].values[1][-2]*0.01



#회사명 : 회사종목코드

company_name = input("기업 가치를 알고 싶은 회사: ")
#####################################
#company_codes.xlsx 의 경로설정 필요#
#####################################
company_codes = pd.read_excel(r'C:\Users\조두환\Desktop\책\01\001\기업가치평가분석\company_codes.xlsx',converters={'종목코드':str})
name_input = company_name
while len(company_codes[company_codes.회사명 ==  name_input]) == 0:
    print('The company name entered does not exist. | 해당 이름의 회사명이 존재하지 않습니다. \nPlease find below for suggestions and re-type the company name | 아래 회사명 중 하나를 찾으시나요? 다시 입력해주세요.\n')
    for row in company_codes.회사명:
        if row.find(name_input) != -1:
            print(row)
    name_input = input()
company_code = company_codes[company_codes.회사명 == name_input].종목코드.iloc[0]
print("Company name | 회사명: "+name_input+"\nCompany code | 종목코드: "+company_code)



#재무제표 네이버 크롤링

python_dict = literal_eval("{'a': 1}")

rpt=[0,1,2] #0 -- IS / 1 -- BS / 2 -- CF
freq=0 #0 -- yearly/ 1 -- quarter

pat_enc = re.compile("encparam: '(.*)'")
#pat_id = re.compile("id: '([a-zA-Z0-9]*)' ?", re.IGNORECASE)
url = "https://companyinfo.stock.naver.com/v1/company/c1030001.aspx?cmp_cd={}&cn".format(company_code)
html = requests.get(url).text
encparam = pat_enc.search(html).group(1)


def FinancialStatement(company_code, freq, num, encparam):
    profitUrl = "https://companyinfo.stock.naver.com/v1/company/cF3002.aspx?cmp_cd={}&frq={}&rpt={}&finGubun=IFRSS&frqTyp={}&cn=&encparam={}".format(company_code, freq, num, freq, encparam)
    headers = {
      "Referer": "HACK"
    }
    #print(profitUrl)
    profitHtml = requests.get(profitUrl, headers=headers).text
    profitSoup = bs(profitHtml, "html5lib")
    profitSoup = profitSoup.select("body")[0].text
    profitJson = json.loads(profitSoup)
    
    #정규표현식 - yet
    #removeDotsAndAsterisk = re.sub("(\.*|\**)","",str(profitJson))
    #eval_fix = re.sub("(\,\s\'QOQ_COMMENT\'(.+)\s1\})","}",removeDotsAndAsterisk) 
    #profitJson1=literal_eval(eval_fix)

    
    #profitDate = profitJson["YYMM"]
    profitData = profitJson["DATA"]
    Dframe=pd.DataFrame(profitData, dtype='float')
    Dframe.fillna(0, inplace=True)
    return Dframe

def Merge(FS, FS_quarter):
    FS_left=FS[['ACC_NM','DATA2','DATA3','DATA4','DATA5']]
    FS_right=FS_quarter[['DATA5']]
    FS_right.columns=['DATA6']
    FS_right=FS_right*(4/3)
    FinancialStatement = FS_left.join(FS_right, how='outer')
    
    return FinancialStatement
    

for num in rpt:
    if(num==0):
        IS=FinancialStatement(company_code, freq, num, encparam)
        freq+=1
        IS_quarter=FinancialStatement(company_code, freq, num, encparam)
        freq=0
        IS=Merge(IS,IS_quarter)
        IS.rename({'ACC_NM':'포괄손익계산서', 'DATA2':'2014/12', 'DATA3':'2015/12', 'DATA4':'2016/12', 'DATA5':'2017/12', 'DATA6':'2018/12'}, axis=1, inplace=True)
        IS_timesMillion=IS.select_dtypes(exclude=['object']) * 100000000
        IS[IS_timesMillion.columns] = IS_timesMillion
        
        
    elif(num==1):
        BS=FinancialStatement(company_code, freq, num, encparam)
        freq+=1
        BS_quarter=FinancialStatement(company_code, freq, num, encparam)
        freq=0
        BS=Merge(BS,BS_quarter)
        BS.rename({'ACC_NM':'재무상태표', 'DATA2':'2014/12', 'DATA3':'2015/12', 'DATA4':'2016/12', 'DATA5':'2017/12', 'DATA6':'2018/12'}, axis=1, inplace=True)
        BS_timesMillion=BS.select_dtypes(exclude=['object']) * 100000000
        BS[BS_timesMillion.columns] = BS_timesMillion
        
        
    elif(num==2):
        CF=FinancialStatement(company_code, freq, num, encparam)
        freq+=1
        CF_quarter=FinancialStatement(company_code, freq, num, encparam)
        freq=0
        CF=Merge(CF,CF_quarter)
        CF.rename({'ACC_NM':'현금흐름표', 'DATA2':'2014/12', 'DATA3':'2015/12', 'DATA4':'2016/12', 'DATA5':'2017/12', 'DATA6':'2018/12'}, axis=1, inplace=True)
        CF_timesMillion=CF.select_dtypes(exclude=['object']) * 100000000
        CF[CF_timesMillion.columns] = CF_timesMillion

        
        
#총 차입금

ST_borrowing=BS.loc[BS['재무상태표']=='....단기차입금']
LT_borrowing=BS.loc[BS['재무상태표']=='....장기차입금']
for idx, col in enumerate(ST_borrowing.columns):
    if(idx==0):
        pass
    elif col in LT_borrowing.columns:
        ST_borrowing[col]=ST_borrowing[col].values+LT_borrowing[col].values
Total_borrowing=ST_borrowing.reset_index().iloc[0][2:]
BW=Total_borrowing.tolist()



#Beta

html = "https://companyinfo.stock.naver.com/v1/company/c1010001.aspx?cmp_cd=005930&cn="
BetaCrawler = pd.read_html(html)
beta=float(BetaCrawler[1].values[5][1])


#보유 주식량(리펙토링 필요) -- 재무제표나 Beta 크롤링 시 같이 작업이 끝나게 수정 필요

def get_date_str(s):
    date_str = ''
    r = re.search("\d{4}/\d{2}", s)
    if r:
        date_str = r.group()
        date_str = date_str.replace('/', '-')

    return date_str

'''
* code: 종목코드
* fin_type(종류): 0=주재무제표(기본), 1=GAAP개별, 2=GAAP연결, 3=IFRS별도, 4=FRS연결
* freq_type(기간): Y=년(기본), Q=분기
'''
def get_finstate_naver(code, fin_type='0', freq_type='Y'):
    # encparam, encid  추출
    url_tmpl = 'http://companyinfo.stock.naver.com/v1/company/c1010001.aspx?cmp_cd=%s'
    url = url_tmpl % ( code)

    html_text = requests.get(url).text
    if '기업재무정보 접속장애' in html_text:
        return None

    encparam = re.findall ("encparam: '(.*?)'", html_text)[0]
    encid = re.findall ("id: '(.*?)'", html_text)[0]

    #  재무데이터 표 추출
    url_tmpl = 'http://companyinfo.stock.naver.com/v1/company/ajax/cF1001.aspx?' \
                    'cmp_cd=%s&fin_typ=%s&freq_typ=%s&encparam=%s&id=%s'

    url = url_tmpl % (code, fin_type, freq_type, encparam, encid)

    header = {
        'Referer': 'https://companyinfo.stock.naver.com/v1/company/c1010001.aspx',
    }

    html_text = requests.get(url, headers=header).text
    dfs = pd.read_html(StringIO(html_text))
    df = dfs[1]
    if df.iloc[0,0].find('해당 데이터가 존재하지 않습니다') >= 0:
        return None
    
    cols = list(df.columns)
    new_cols = []
    new_cols.append(cols[0][0])
    new_cols += [c[1] for c in cols[:-1]]
    df.columns = new_cols
    df.set_index('주요재무정보', inplace=True)
    df.columns = [get_date_str(x) for x in df.columns]
    
    return df
df = get_finstate_naver(company_code, fin_type='3', freq_type='Q')
NO=df['2018-09'][-1]



# 무위험 이자율 계산 
url = "http://www.index.go.kr/potal/main/EachDtlPageDetail.do?idx_cd=1073" #e나라지표 사이트에서 국고채 평균 10년 금리 추출
html = urlopen(url)
source = html.read() 
html.close() 
soup = bs(source, "html5lib")
Rf = float(soup.select("tbody > tr > td[data-id='td_107301_13']")[2].string)/100 #10년물 국공채 평균 이자율 #매달


# kospi 5년치 시장수익률(Rm)
url = 'https://finance.naver.com/sise/sise_index_day.nhn?code=KPI200'
html = urlopen(url)
source = bs(html.read(), "html.parser")

maxPage=source.find_all("table",align="center")
mp = maxPage[0].find_all("td",class_="pgRR")
mpNum = int(mp[0].a.get('href')[-3:])

pageNum, Kospi200diff=[1,205], []
for i in range(len(pageNum)):
    url = 'https://finance.naver.com/sise/sise_index_day.nhn?code=KPI200' +'&page='+ str(pageNum[i])
    html = urlopen(url)
    source = bs(html.read(), "html.parser")
    srlists=source.find_all("tr")
    isCheckNone = None
    #print(float(srlists[2].find("td",class_="number_1").text))
    Kospi200diff.append(float(srlists[2].find("td",class_="number_1").text))

Rm=(Kospi200diff[0]-Kospi200diff[1])*0.01
'''
Rm-Rf가 마이너스 값이 나올경우 무성장 가정
if(Rm-Rf < 0):
    Rm=0
    Rf=0
'''
#  주식 수익률 계산
Re = Rf + (beta * (Rm - Rf))


#CFO
g1=CF.loc[(CF['현금흐름표'] == '영업활동으로인한현금흐름')]
CFO=g1.iloc[0,1:]

#INT
g2=CF.loc[(CF['현금흐름표'] == '........금융비용')]
INT=g2.iloc[0,1:]

#TAXRATE

g3=IS.loc[(IS['포괄손익계산서'] == '법인세비용')]
g3=g3.iloc[0,1:]
g4=IS.loc[(IS['포괄손익계산서'] == '법인세비용차감전계속사업이익')]
g4=g4.iloc[0,1:]

corporateTaxRate=g3/g4

#세율이 마이너스일 경우는 기업이 국가에게 환급받은 경우 -> 우리나라 법인세율(20%) 적용
for i in range(len(corporateTaxRate)):
    if(corporateTaxRate[i] < 0):
        corporateTaxRate[i]= 0.2 
TAXRATE=corporateTaxRate

#CAPEX

g5=CF.loc[(CF['현금흐름표'] == '투자활동으로인한현금흐름')]
CAPEX=g5.iloc[0,1:]

a=[CFO,INT,TAXRATE,CAPEX]
b=pd.DataFrame(a)
if(b.iloc[0,[4]].values[0]+b.iloc[3,[4]].values[0] < 0):
    b.iloc[3,[4]]=0
c=b.iloc[0]+b.iloc[1]*(1-b.iloc[2])+b.iloc[3] #FCFF계산
FCFF=b.append([c])
FCFF=FCFF.append(FCFF.iloc[4].pct_change())
FCFF.index = ["CFO","INT","TAXRATE","CAPEX", "FCFF","FCFF_growthRate"]


#경우의 수 1 : 해당기업의 성장률이 변동이 심한경우, gdp성장률로 대체
#FCFF_G = FCFF.iloc[5,[3,4]].mean() 
FCFF_G =GDPgrowthRate 


#10년 예측 재무제표

FUTURE=['2019','2020','2021','2022','2023','2024','2025','2026','2027','2028']
ExpFCFF=FCFF.iloc[4,[4]]
FUTUREFCFF=[]
for j in range(10):
    ExpFCFF=(FCFF_G + 1)*ExpFCFF
    FUTUREFCFF.append(ExpFCFF.round(2))

FUTUREFCFF1=[{FUTURE[i]:FUTUREFCFF[i] for i in range(10)}] 

F_FCFF=pd.DataFrame(FUTUREFCFF1)


# r - WACC 계산 
eRatio = BS.loc[(BS['재무상태표'] == '자본총계')].values[0][1:][-1]/BS.loc[(BS['재무상태표'] == '자산총계')].values[0][1:][-1] # 자본 비율
dRatio = 1 - eRatio # 부채비율
#iRate = FCFF.iloc[1,4] / BW[-1] # 차입금/이자율
iRate=.0421
taxR = FCFF.iloc[2].mean() # 법인 세율
#WACC =eRatio *  Re  + iRate*(1-taxR)*dRatio
WACC=0.04 # 이자율이 너무 나가서 일단 4%로 고정

#if(I_Growth > WACC): #r-g가 0보다 작으면 r만 적용. 분자(1+g)도 g가 0으로 적용. (무성장을 가정하는 경우)
#    I_Growth=0
# Terminal Value 및 FCFF 현재가치 계산


df = F_FCFF
df.iloc[0][-1]=df.iloc[0][-1]+df.iloc[0,9]*(1+I_Growth)/(WACC-I_Growth)
p = pd.DataFrame([np.zeros(10)],columns=['2019','2020','2021','2022','2023','2024','2025','2026','2027','2028'])
df = df.append(p, ignore_index=True)
DiscountedFCFF=[]
for i in range(10):
    DiscountedFCFF.append(df.iloc[0][i]/((1+WACC)**(i+1)))

df.iloc[1]=DiscountedFCFF
EnterpriseValue=df.iloc[1].sum()
EquityValue=EnterpriseValue-BW[-1]
StockPrice = EquityValue / NO

print("***{} 주가에 대한 정보입니다.***".format(company_name))
print("==============================================================")
print("")
print("10년물 국공채 이자율(Rf) : {0:.5f}".format(Rf))
print("KOSPI200 연 수익률(Rm) : {0:.5f}".format(Rm))
print("베타값(beta) : {0:.5f}".format(beta))
print("{0}주식의 예상 수익률(Re) : {1:.2f}".format(company_name, Re))
print("{0}의 차입이자율(iRate) : {1:.2f}".format(company_name, iRate))
print("{0}의 가중평균 투자 수익률(WACC) : {1:.2f}".format(company_name, WACC))
print("{0}의 예상 기업가치(EnterpriseValue) : {1:.2f}".format(company_name, EnterpriseValue[0]))
print("{0}의 주식보유량(NO): {1:.2f}".format(company_name, NO))
print("{0} 주식의 예상 시장가치(EquityValue) : {1:.2f}".format(company_name, EquityValue[0]))
print("==============================================================")
print("")
print("{0} 주식의 예상 주당가치(StockPrice) : {1:.2f}".format(company_name, StockPrice[0]))
#print("{0} 주식의 현재 거래가격(todayP) : {1:.2f}".format(company_name, todayP)) #현재 주가 crawling 구현 필요
print("==============================================================")
#print("")
#print("{0} 주식에 대한 의견 : {1}".format(comp, opinion))


기업 가치를 알고 싶은 회사: 현대자동차
Company name | 회사명: 현대자동차
Company code | 종목코드: 005380


C:\anaconda3\lib\site-packages\ipykernel_launcher.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


***현대자동차 주가에 대한 정보입니다.***

10년물 국공채 이자율(Rf) : 0.02210
KOSPI200 연 수익률(Rm) : 0.07170
베타값(beta) : 1.40000
현대자동차주식의 예상 수익률(Re) : 0.09
현대자동차의 차입이자율(iRate) : 0.04
현대자동차의 가중평균 투자 수익률(WACC) : 0.04
현대자동차의 예상 기업가치(EnterpriseValue) : 19534886833043.38
현대자동차의 주식보유량(NO): 213668187.00
현대자동차 주식의 예상 시장가치(EquityValue) : 15678932166376.71

현대자동차 주식의 예상 주당가치(StockPrice) : 73379.82
